# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Weights and Biases

In [2]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8MB 14.9MB/s 
     |████████████████████████████████| 163kB 56.5MB/s 
     |████████████████████████████████| 102kB 15.9MB/s 
     |████████████████████████████████| 133kB 48.9MB/s 
     |████████████████████████████████| 102kB 15.1MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 


# Configure wandb sweep

In [3]:
from wandb.keras import WandbCallback
import wandb

In [5]:
# Configure the sweep
sweep_config = {
    'method': 'grid',
    'name' : 'Neuron & LR Sweep - Batch Size 64',
    'parameters': {
        'learning_rate': {
            'values': [5e-4, 1e-5]
        },
        'fc1_num_neurons': {
            'values': [512, 1024]
        },
        'fc3_num_neurons': {
            'values': [256, 512]
        },
    }
}

In [6]:
WANDB_API_KEY='002b3552fd78e3559e38c7317ef0d9eefd0c7c6d' 
sweep_id = wandb.sweep(sweep_config, project="nn_project_tuning")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: r4y8hd2a
Sweep URL: https://wandb.ai/burntice/nn_project_tuning/sweeps/r4y8hd2a


# Model

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf

# Load dataset as dataframe
base_path = '/content/drive/My Drive/Y4 Sem 1/NN Project/'
df = pd.read_csv('/content/drive/My Drive/Y4 Sem 1/NN Project/aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_val_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# function to call for hyperparameter sweep
def train():
    # Fix seed
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # default hyperparameters
    defaults = {
        'epochs': 20,
        'batch_size': 64,
        'learning_rate': 1e-5,
        'weight_decay': 0.01,
        'fc1_num_neurons': 1024,
        'fc2_num_neurons': 512,
        'fc3_num_neurons': 256,
        'dropout1_rate': 0,
        'dropout2_rate': 0,
        'dropout3_rate': 0,
        'seed': seed,
        'optimizer': 'adam',
        'hidden_activation': 'relu',
        'output_activation': 'sigmoid',
        'loss_function': 'binary_crossentropy',
        'metrics': ['accuracy'],
    }

    wandb.init(
        config=defaults,
        name='Hyperparameter Tuning',
        project='nn_project',
    )
    config = wandb.config

    # Define model
    mobile_net_v2 = tf.keras.applications.MobileNetV2(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_shape=(224,224,3),
    )

    fc1 = tf.keras.layers.Dense(
        config.fc1_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc2 = tf.keras.layers.Dense(
        config.fc2_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc3 = tf.keras.layers.Dense(
        config.fc3_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    bn1 = tf.keras.layers.BatchNormalization()
    bn2 = tf.keras.layers.BatchNormalization()
    bn3 = tf.keras.layers.BatchNormalization()
    bn4 = tf.keras.layers.BatchNormalization()

    dropout1 = tf.keras.layers.Dropout(rate=config.dropout1_rate)
    dropout2 = tf.keras.layers.Dropout(rate=config.dropout2_rate)
    dropout3 = tf.keras.layers.Dropout(rate=config.dropout3_rate)

    model = tf.keras.models.Sequential([
        mobile_net_v2,
        tf.keras.layers.Flatten(),
        bn1,
        fc1,
        dropout1,
        bn2,
        fc2,
        dropout2,
        bn3,
        fc3,
        dropout3,
        bn4,
        tf.keras.layers.Dense(1, activation=config.output_activation),
    ])

    # Compile model 
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
        loss=config.loss_function,
        metrics=config.metrics,
    )

    # Load images into keras image generator 
    datagen_train = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_val = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_test = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )

    train_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    val_generator = datagen_train.flow_from_dataframe(
        dataframe=val_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    test_generator = datagen_test.flow_from_dataframe(
        dataframe=test_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=config.epochs,
        callbacks=[WandbCallback()],
    )

# Run agent

In [15]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: d165tp26 with config:
wandb: 	fc1_num_neurons: 512
wandb: 	fc3_num_neurons: 512
wandb: 	learning_rate: 1e-05
wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


9412608/9406464 [==============================] - 0s 0us/step
Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 2160s 18s/step - loss: 18.2134 - accuracy: 0.5901 - val_loss: 18.0400 - val_accuracy: 0.6110
Epoch 2/20
122/122 [==============================] - 131s 1s/step - loss: 17.7963 - accuracy: 0.7494 - val_loss: 17.7579 - val_accuracy: 0.7037
Epoch 3/20
122/122 [==============================] - 130s 1s/step - loss: 17.4799 - accuracy: 0.8169 - val_loss: 17.5003 - val_accuracy: 0.7514
Epoch 4/20
122/122 [==============================] - 131s 1s/step - loss: 17.1872 - accuracy: 0.8624 - val_loss: 17.2448 - val_accuracy: 0.7683
Epoch 5/20
122/122 [==============================] - 130s 1s/step - loss: 16.9067 - accuracy: 0.8993 - val_loss: 16.9770 - val_accuracy: 0.7914
Epoch 6/20
122/122 [==============================] - 132s 1s/step - loss: 16.6279 - accuracy: 0

epoch,19
loss,12.40005
accuracy,0.99744
val_loss,12.51972
val_accuracy,0.88262
_step,19
_runtime,4754
_timestamp,1605951410
best_val_loss,12.51972
best_epoch,19


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▁▁
accuracy,▁▄▅▆▆▇▇▇▇███████████
val_loss,██▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
val_accuracy,▁▃▅▅▆▆▆▇▇███████████
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Agent Starting Run: 1a1fnjfu with config:
wandb: 	fc1_num_neurons: 1024
wandb: 	fc3_num_neurons: 256
wandb: 	learning_rate: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: t1tgubzv with config:
wandb: 	fc1_num_neurons: 1024
wandb: 	fc3_num_neurons: 512
wandb: 	learning_rate: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 135s 1s/step - loss: 23.8799 - accuracy: 0.6342 - val_loss: 23.6918 - val_accuracy: 0.6591
Epoch 2/20
122/122 [==============================] - 131s 1s/step - loss: 23.3822 - accuracy: 0.7822 - val_loss: 23.3264 - val_accuracy: 0.7160
Epoch 3/20
122/122 [==============================] - 131s 1s/step - loss: 22.9655 - accuracy: 0.8437 - val_loss: 22.9804 - val_accuracy: 0.7412
Epoch 4/20
122/122 [==============================] - 132s 1s/step - loss: 22.5682 - accuracy: 0.8854 - val_loss: 22.6215 - val_accuracy: 0.7581
Epoch 5/20
122/122 [==============================] - 132s 1s/step - loss: 22.1653 - accuracy: 0.9182 - val_loss: 22.2361 - val_accuracy: 0.7745
Epoch 6/20
122/122 [==============================] - 132s 1s/step - loss: 21.7679 - accuracy: 0.9348 - val_loss: 21.8295 - val_accuracy: 0.7965
Epoch 7/20
122/1

epoch,19
loss,15.40696
accuracy,0.99859
val_loss,15.41708
val_accuracy,0.8939
_step,19
_runtime,2773
_timestamp,1605962167
best_val_loss,15.41708
best_epoch,19


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,██▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
accuracy,▁▄▅▆▆▇▇▇████████████
val_loss,██▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
val_accuracy,▁▃▃▄▄▅▆▆▇▇▇▇▇▇██████
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
_timestamp,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
